# Estimating a Yield Curve Model

## Loading Packages

In [1]:
using Optim

include("jlFiles/printmat.jl")

printmatDate

In [2]:
using Plots

backend = "gr"              #"gr" (default), "pyplot" 

if backend == "pyplot"
    pyplot(size=(600,400))
else    
    gr(size=(600,400))
end

Plots.GRBackend()

## Loading Data

In [3]:
x   = readdlm("Data/USCMRatesPs.csv",',',skipstart=1)
y   = convert(Array{Float64},x[:,3:end])   #interest rates, 3m to 10y, skip FFR
ymd = x[:,1]                               #strings, "2012-03-01"


df = Dates.DateFormat("yyyy-mm-dd")
dN = Date.(ymd,df)                  #Julia dates 

y[:,1] = ((1 .- 0.25*y[:,1]/100).^(-1/0.25) - 1)*100    #discount basis->effective
y[:,2] = ((1 .- 0.50*y[:,2]/100).^(-1/0.5)  - 1)*100

m      = [0.25 0.5 1 3 5 7 10]      #time to maturity (in years)
mMonth = round.(Int,m*12)           #maturites, in months (integers)
y      = log.(1 .+ y/100)           #continuously compounded interest rates
y      = y/12                       #interest rates per month (the period length of data)

(T,n) = size(y)

(570, 7)

## Establishing Notation for an Affine Yield Curve Model


An affine yield curve model implies that the yield on an $n$-period discount
bond is

$$
\begin{equation}
y_{nt}=a_{n}+b_{n}^{\prime}x_{t}\nonumber
\end{equation}
$$

where $x_{t}$ is an $K\times1$ vector of state variables. Here $a_{n}$ is a
scalar and $b_{n}$ is a $K\times1$ vector (so $b_{n}^{\prime}x_{t}$ is a scalar). 

Transpose to get
$$
y_{nt}=a_{n}+\underset{1\times K}{\underbrace{x_{t}^{\prime}}}
\underset{K\times1}{\underbrace{b_{n}}}
$$

As an example, consider two different maturities ($n$), denoted $1$ and $2$.
The row vector is then

$$
\begin{bmatrix}
y_{1t} & y_{2t}
\end{bmatrix} =
\begin{bmatrix}
a_{1} & a_{2}
\end{bmatrix}
+x_{t}^{\prime}
\begin{bmatrix}
b_{1} & b_{2}
\end{bmatrix}.
$$

Now, stack observations ($t=1$ to $T$) vertically

$$
Y_{T\times n}=\mathbf{1}_{T\times1}\otimes a_{1\times n}+X_{T\times K}b_{K\times n},
$$

where $\mathbf{1}_{T\times1}$ is a $T\times1$ vector of ones and $\otimes$ is
the Kronecker product.

An example of the Kronecker product:
$$
\mathbf{1}_{2\times1}\otimes
\begin{bmatrix}
1 & 2 & 3
\end{bmatrix} =
\begin{bmatrix}
1 & 2 & 3\\
1 & 2 & 3
\end{bmatrix}.
$$

### From Model Parameters to $a_{n}$ and $b_{n}$

The following function VasicekABFn.jl takes the
Vasicek model parameters as inputs and calculates the $a_{n}$ and $b_{n}$ values.

In [4]:
function VasicekABFn(lambda,mu,rho,sigma,nMo,nMu,yo)

  nMax = maximum([nMo;nMu])    #longest maturity to calculate (a,b) for
  Nvec = 1:nMax

  A = fill(NaN,(1,nMax))              #recursive solution of AR(1) model
  B = fill(NaN,(1,nMax))
  B[1] = 1 + 0*rho
  A[1] = 0 + 0*(1-rho)*mu - (lambda+0)^2*sigma^2/2
  for n = 2:nMax
    B[n] = 1 + B[n-1]*rho
    A[n] = A[n-1] + B[n-1]*(1-rho)*mu - (lambda+B[n-1])^2*sigma^2/2
  end

  a  = A./Nvec'
  b  = B./Nvec'

  ao = a[:,nMo][1]                       #for "observed" maturity
  bo = b[:,nMo][1]                       #[1] to make sure it is a scalar

  xt    = (yo.-ao)/bo                    #value of state variable x(t)
  au    = a[:,nMu]
  bu    = b[:,nMu]
  yuHat = au .+ xt*bu                   #Txn, fitted values of yu

  return ao,bo,xt,au,bu,yuHat

end

VasicekABFn (generic function with 1 method)

### Testing the VasicekABFn Function

In [5]:
vvo = 1                               #indices of yields that are observed without errors
vvu = 2:n                             #indices of yields with observation errors

yo = y[:,vvo]                         #observable yields
yu = y[:,vvu]                         #unobservable yields
nMo = mMonth[vvo]                     #maturity of yo
nMu = mMonth[vvu]                     #maturity of yu

#just testing the Vasicek functions
(ao,bo,xt,au,bu,yuHat) = VasicekABFn(1,0.5,0.9,0.02,nMo,nMu,yo)

println("ao and bo: ")
printmat([ao bo])

ao and bo: 
     0.047     0.903



In [6]:
xTicksLoc = Dates.value.(Date(1970):Dates.Year(10):Date(2010))
xTicksLab = ["1970";"1980";"1990";"2000";"2010"]               #crude way of getting the tick marks right

plot(dN,xt,legend=false,xticks=(xTicksLoc,xTicksLab))
title!("xt (implied by ao and bo, using specific parameter values)")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1970 
 
 
 1980 
 
 
 1990 
 
 
 2000 
 
 
 2010 
 
 
 -0.051 
 
 
 -0.048 
 
 
 -0.045 
 
 
 -0.042 
 
 
 -0.039 
 
 
 xt (implied by ao and bo, using specific parameter values) 
 
<polyline clip-path="url(#clip02)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 52.3228,229.818 53.2679,230.239 54.2129,227.921 55.1274,229.818 56.0724,224.759 56.987,212.521 57.932,211.465 58.877,227.078 59.7306,237.61 60.6756,240.136 
 61.5901,233.188 62.5352,236.558 63.4497,241.399 64.3947,242.241 65.3397,248.34 66.2543,252.754 67.1993,265.982 68.1138,274.579 69.0589,283.586 70.0039,299.064 
 70.8574,305.748 71.8025,295.719 72.717,289.864 73.662,277.094 74.5766,263.464 75.5216,273.112 76.4666,278.351 77.3811,283.167 78.3262,288.19 79.2407,292.583 
 80.1857,305.748 81.1307,309.505 82.0148,298.437 82.9598,298.855 83.8743,299.273 84.8194,294.674 85.7339,293.211 86.6789,292.374 87.6239,278.979 88.5385,277.303 
 89.4835,276.465 90.398,270.386 91.343,263.254 92.2881,259.265 93.1416,248.971 94.0867,245.396 95.0012,243.293 95.9462,225.813 96.8607,208.508 97.8058,194.553 
 98.7508,202.591 99.6653,225.181 100.61,212.31 101.525,220.33 102.47,213.577 103.415,227.289 104.268,209.564 105.214,201.745 106.128,203.859 107.073,210.831 
 107.988,218.22 108.933,188.414 109.878,207.452 110.792,220.119 111.737,219.908 112.652,226.656 113.597,245.396 114.542,261.365 115.395,261.575 116.34,259.055 
 117.255,267.031 118.2,264.723 119.114,248.13 120.059,241.609 121.004,242.03 121.919,251.703 122.864,261.785 123.779,262.624 124.724,274.579 125.669,274.369 
 126.553,271.854 127.498,274.789 128.412,267.66 129.357,263.254 130.272,267.031 131.217,268.918 132.162,270.177 133.076,273.531 134.021,277.094 134.936,285.47 
 135.881,279.817 136.826,278.769 137.68,280.236 138.625,281.492 139.539,272.692 140.484,271.435 141.399,267.87 142.344,261.575 143.289,254.855 144.203,247.499 
 145.148,248.76 146.063,249.391 147.008,241.609 147.953,241.399 148.806,244.765 149.751,244.765 150.666,242.241 151.611,235.505 152.526,229.607 153.471,228.132 
 154.416,211.887 155.33,208.93 156.275,195.188 157.19,185.872 158.135,180.151 159.08,180.787 159.933,177.395 160.878,177.819 161.793,174.637 162.738,186.296 
 163.652,182.482 164.597,176.546 165.542,160.838 166.457,130.184 167.402,128.265 168.316,122.93 169.262,123.784 170.207,105.408 171.091,55.2026 172.036,98.1324 
 172.95,196.457 173.895,228.343 174.81,207.452 175.755,184.813 176.7,160.625 177.614,131.89 178.559,86.7777 179.474,48.9593 180.419,59.0754 181.364,64.0214 
 182.217,94.7062 183.162,87.6352 184.077,31.4961 185.022,65.3111 185.937,60.581 186.882,48.5286 187.827,65.956 188.741,90.85 189.686,148.079 190.601,148.292 
 191.546,117.806 192.491,92.1356 193.344,109.258 194.289,108.83 195.204,121.863 196.149,113.746 197.063,137.645 198.008,194.342 198.953,210.409 199.868,214.843 
 200.813,207.24 201.728,209.987 202.673,211.676 203.618,206.395 204.471,201.322 205.416,204.282 206.331,204.704 207.276,192.013 208.19,185.872 209.135,180.363 
 210.08,187.567 210.995,195.188 211.94,192.648 212.854,187.567 213.799,189.685 214.744,185.661 215.628,176.546 216.574,172.94 217.488,169.969 218.433,169.12 
 219.348,163.812 220.293,156.375 221.238,158.5 222.152,171.879 223.097,195.823 224.012,207.452 224.957,213.788 225.902,203.013 226.755,197.727 227.7,209.775 
 228.615,219.697 229.56,230.871 230.474,228.132 231.419,226.867 232.364,227.71 233.279,226.446 234.224,224.759 235.139,227.71 236.084,228.343 237.029,228.553 
 237.882,239.084 238.827,249.601 239.742,247.709 240.687,246.447 241.601,254.435 242.546,260.735 243.491,267.45 244.406,268.079 245.351,264.513 246.265,260.735 
 247.21,262.834 248.155,259.475 249.009,259.475 249.954,258.426 250.869,258.006 251.814,257.796 252.728,257.376 253.673,250.022 254.618,242.451 255.533,248.13 
 256.478,257.376 257.392,255.695 258.337,254.855 259.282,258.006 260.166,257

## The Likelihood Function

To formulate the log-liklihood, the VasicekTsCsFn
first takes the parameter vectors and transforms the parameters (scaling,
making sure that $-1\leq\rho\leq1$, etc) and the calls on the function
VasicekABFn (see above).


From the previous output, the code in VasicekTsCsFn moves on to
define the 1-step ahead forecast errors of $y_{ot}$ as

$$
v_{t}=y_{ot}-E_{t-1}y_{ot}
$$

and its variance $S=b_{o}^{\prime}\sigma^{2}b_{o}$. The contribution to the
likelihood function in $t$ is proportional to $v_{t}^{\prime}S^{-1}v_{t}$.
Then, the code defines the cross-sectional errors (for $y_{ut}$) as

$$
u_{t}=y_{ut}-\hat{y}_{ut} \ \text{, where } \ \hat{y}_{ut}=a_{u}+b_{u}^{\prime}x_{t},
$$

The contribution to the likelihood function is $u_{t}^{\prime}\Omega^{-1}
u_{t}$, where the code assumes that $\Omega$ is diagonal,
$
\Omega=I\omega^{2},
$
where $\omega$ is one of the parameters estimated. 

The function VasicekTsCsLossFn creates
the value of the function that will be minimized.

In [7]:
function VasicekTsCsFn(par,yo,yu,nMo,nMu)
#VasicekTsCsFn    Loss function for estimating the Vasicek model using both
#                 time series and cross sectional information

  J = length(nMu)
  T = size(yo,1)

  lambda  = par[1]*100
  mu      = par[2]/1200
  p       = 1 - 2/(1+exp(par[3]))          #inverse: par[3] = log((1+p)/(1-p))
  s2      = (par[4]/1200)^2
  omega_i = abs(par[5]/1200)

  if length(nMo) != 1
    error("yo must be a single yield")
  end

  (ao,bo,xt,au,bu,yuHat) = VasicekABFn(lambda,mu,p,sqrt(s2),nMo,nMu,yo)

  Et1xt    = (1-p)*mu .+ p*[NaN;xt[1:end-1]]   #E(t-1)x(t)
  Et1xt[1] = mu
  Et1yo    = ao .+ Et1xt*bo             #E(t-1)yo(t)
  v        = yo - Et1yo                 #Tx1, forecast error of yo
  S        = bo's2*bo                   #variance of forecast error of yo
  S_1      = inv(S*1000)*1000           #1000/1000 improves the precision a bit
  LLo_t    = -0.5*log(2*pi) .- 0.5*log(det(S)) .- 0.5*sum(v*S_1.*v,2)   #Tx1

  u        = yu - yuHat                 #TxL, fitted errors of yu
  Omega    = eye(J)*omega_i^2
  Omega_1  = inv(Omega)                  #covariance matrix of u
  LLu_t    = -0.5*J*log(2*pi) .- 0.5*log(det(Omega)) .- 0.5*sum(u*Omega_1.*u,2)  #Tx1

  LL_t    = LLo_t + LLu_t                #Tx1, log likelihood(t), sum of TS and CS
  
  return LL_t,yuHat,u,xt

end


function VasicekTsCsLossFn(par,yo,yu,nMo,nMu)
  LL_t,   = VasicekTsCsFn(par,yo,yu,nMo,nMu)
  MinusLL = -sum(LL_t[2:end])            #to be minimized 
  return MinusLL
end

VasicekTsCsLossFn (generic function with 1 method)

## Trying the Loss Function

In [8]:
#notice: important that the elements of the parameter vector have similar magnitude
#lambda/100,mu*1200,p = 1 - 2/(1+exp(par(3))) so 5.6 -> p = 0.993,sigma*1200,omega*1200
par0 = [-2.3;10;5.5;0.5;0.8]

MinusLL = VasicekTsCsLossFn(par0,yo,yu,nMo,nMu)

printlnPs("The negative of the log-likelihood function at the starting guess: ",MinusLL)

The negative of the log-likelihood function at the starting guess: -23764.802


## Minimizing the Loss Function

In [9]:
Sol = optimize(par->VasicekTsCsLossFn(par,yo,yu,nMo,nMu),par0)
par1 = Optim.minimizer(Sol)

println("\nStarting guess   estimate")
printmat([par0 par1])

#converting back to model parameters 
(lambda,mu,rho,s2) = (par1[1]*100,par1[2]/1200,1 - 2/(1+exp(par1[3])),par1[4]/1200)
println("Estimated (lambda,mu,rho,s2):")
printmat([lambda mu rho s2])


Starting guess   estimate
    -2.300    -2.475
    10.000     9.270
     5.500     5.856
     0.500     0.502
     0.800     0.779

Estimated (lambda,mu,rho,s2):
  -247.453     0.008     0.994     0.000



In [10]:
MinusLL = VasicekTsCsLossFn(par1,yo,yu,nMo,nMu)
println("The negative of the log-likelihood function at the point estimate: ",MinusLL)

(LL_t,yuHat,u,xt) = VasicekTsCsFn(par1,yo,yu,nMo,nMu)

yhatTsCs = [yo yuHat]                      #fitted yields

                                            #average yield curve
(ao,bo,xt,au,bu,yuHatAvg) = VasicekABFn(lambda,mu,rho,s2,nMo,nMu,mean(yo))
yuHatAvg = [mean(yo) yuHatAvg]             #also yo

println()

The negative of the log-likelihood function at the point estimate: -23860.919153719806



In [13]:
yy = y[:,[3,6]] - yhatTsCs[:,[3,6]]  

plot(dN,12*yy[:,1],label="1 year")
plot!(dN,12*yy[:,2],line=(:dot,2),label="7 years")
plot!(ylims=(-0.04,0.04),xticks=(xTicksLoc,xTicksLab))
title!("Pricing errors, Vasicek")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1970 
 
 
 1980 
 
 
 1990 
 
 
 2000 
 
 
 2010 
 
 
 -0.04 
 
 
 -0.02 
 
 
 0.00 
 
 
 0.02 
 
 
 0.04 
 
 
 Pricing errors, Vasicek 
 
<polyline clip-path="url(#clip02)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 45.7332,197.463 46.69,199.009 47.6468,192.515 48.5728,195.863 49.5296,196.129 50.4556,209.774 51.4124,214.699 52.3692,203.414 53.2334,206.945 54.1902,198.207 
 55.1162,184.636 56.073,185.804 56.9989,194.04 57.9558,197.16 58.9126,194.869 59.8385,198.04 60.7953,208.6 61.7213,212.033 62.6781,211.443 63.6349,208.158 
 64.4991,202.906 65.4559,197.982 66.3819,179.419 67.3387,180.783 68.2647,192.137 69.2215,182.338 70.1783,187.617 71.1042,198.32 72.0611,197.998 72.987,191.976 
 73.9438,178.486 74.9006,171.285 75.7957,177.235 76.7525,164.48 77.6785,176.775 78.6353,174.181 79.5613,175.917 80.5181,176.792 81.4749,181.854 82.4008,185.25 
 83.3576,197.158 84.2836,199.267 85.2404,198.655 86.1972,194.561 87.0614,188.755 88.0183,195.999 88.9442,198.65 89.901,217.193 90.827,209.161 91.7838,220.399 
 92.7406,224.33 93.6666,214.867 94.6234,234.141 95.5493,229.909 96.5061,237.58 97.463,231.484 98.3272,232.11 99.284,213.743 100.21,203.123 101.167,193.349 
 102.093,173.23 103.05,211.535 104.006,192.284 104.932,199.154 105.889,215.144 106.815,215.484 107.772,196.805 108.729,198.826 109.593,193.125 110.55,166.305 
 111.476,170.727 112.432,179.451 113.358,179.999 114.315,169.569 115.272,166.719 116.198,179.192 117.155,177.314 118.081,171.169 119.038,178.958 119.994,175.317 
 120.889,168.24 121.846,174.061 122.772,169.047 123.729,173.124 124.655,178.42 125.612,182.707 126.569,186.659 127.495,193.71 128.451,195.066 129.377,194.474 
 130.334,189.548 131.291,184.319 132.155,180.123 133.112,180.026 134.038,181.561 134.995,185.737 135.921,187.269 136.878,182.592 137.834,189.74 138.76,186.906 
 139.717,185.155 140.643,183.474 141.6,186.386 142.557,184.406 143.421,178.788 144.378,173.177 145.304,163.497 146.261,166.382 147.186,166.405 148.143,172.572 
 149.1,192.397 150.026,178.626 150.983,172.302 151.909,179.849 152.866,187.153 153.822,192.496 154.687,198.98 155.643,203.198 156.569,209.645 157.526,207.57 
 158.452,212.549 159.409,211.252 160.366,209.56 161.292,209.993 162.249,215.797 163.175,242.337 164.131,237.533 165.088,203.918 165.983,234.466 166.94,242.15 
 167.866,194.054 168.823,178.11 169.749,200.99 170.706,184.337 171.662,183.666 172.588,204.622 173.545,232.991 174.471,282.191 175.428,291.768 176.385,263.299 
 177.249,233.547 178.206,224.849 179.132,268.408 180.089,249.805 181.014,227.284 181.971,214.675 182.928,186.741 183.854,178.61 184.811,174.878 185.737,157.633 
 186.694,163.71 187.65,200.32 188.515,194.984 189.472,194.718 190.397,192.543 191.354,181.358 192.28,164.36 193.237,118.817 194.194,108.718 195.12,159.552 
 196.077,181.263 197.003,185.566 197.959,193.617 198.916,192.449 199.78,197.99 200.737,194.361 201.663,196.665 202.62,192.45 203.546,183.752 204.503,182.048 
 205.46,181.881 206.386,180.134 207.342,180.188 208.268,183.835 209.225,187.76 210.182,190.435 211.077,187.446 212.034,182.706 212.96,159.332 213.917,144.893 
 214.843,157.571 215.799,180.718 216.756,185.658 217.682,184.856 218.639,178.455 219.565,174.137 220.522,173.518 221.479,184.706 222.343,173.03 223.3,176.914 
 224.225,183.708 225.182,187.335 226.108,190.491 227.065,185.479 228.022,182.974 228.948,187.927 229.905,196.528 230.831,198.934 231.787,195.255 232.744,199.625 
 233.608,201.545 234.565,204.024 235.491,199.375 236.448,198.704 237.374,201.109 238.331,202.134 239.288,195.032 240.214,195.792 241.17,199.763 242.096,204.572 
 243.053,203.979 244.01,203.467 244.874,200.625 245.831,183.716 246.757,164.408 247.714,172.886 248.64,178.576 249.597,179.381 250.553,169.063 251.479,160.754 
 252.436,167.294 253.362,162.258 254.319,171.195 255.276,178.913 256.171,177.792 257.128,174.648 258.054,173.901 259.01,178

In [14]:
plot(m',12*yuHatAvg',label="at average x[t] ")
plot!(m',12*mean(y,1)',line=(:dot,2),label="data")
plot!(xlims=(0,11),ylims=(0.05,0.08))
title!("Avg yield curve, Vasicek")
xlabel!("Maturity (years)")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 2.5 
 
 
 5.0 
 
 
 7.5 
 
 
 10.0 
 
 
 0.05 
 
 
 0.06 
 
 
 0.07 
 
 
 0.08 
 
 
 Avg yield curve, Vasicek 
 
 
 Maturity (years) 
 
 
 
 
 
 
 
 at average x[t] 
 
 
 
 data

## Comparing Vasicek Results to OLS (yields on short yield)

In [15]:
bOls = [ones(T) y[:,1]]\y                 #LS of yields on short yield
alfa = bOls[1:1,:]
beta = bOls[2:2,:]
                         #rescaling Vasicek au and bu to be comaparable with OLS on yo
bu_b = bu/bo             #y = au + bu*xt, but xt = (yo-ao)/bo
au_b = au - bu/bo*ao     #y = au + bu*(yo-ao)/bo = y = au - bu*ao/bo + bu/bo*yo
bu_b = [1 bu_b]          #also yo
au_b = [0 au_b]

println()

In [16]:
plot(m',alfa'*1200,label="Vasicek")
plot!(m',au_b'*1200,line=(:dot,2),label="OLS")
plot!(xlims=(0,maximum(m)+1))
title!("Intercept times 1200, Vasicek and OLS")
xlabel!("maturity (years)")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 2.5 
 
 
 5.0 
 
 
 7.5 
 
 
 10.0 
 
 
 0.0 
 
 
 0.5 
 
 
 1.0 
 
 
 1.5 
 
 
 2.0 
 
 
 2.5 
 
 
 Intercept times 1200, Vasicek and OLS 
 
 
 maturity (years) 
 
 
 
 
 
 
 
 Vasicek 
 
 
 
 OLS

In [17]:
plot(m',beta',label="Vasicek")
plot!(m',bu_b',line=(:dot,2),label="OLS")
plot!(xlims=(0,maximum(m)+1))
title!("Slope on short rate, Vasicek and OLS")
xlabel!("maturity (years)")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 2.5 
 
 
 5.0 
 
 
 7.5 
 
 
 10.0 
 
 
 0.75 
 
 
 0.80 
 
 
 0.85 
 
 
 0.90 
 
 
 0.95 
 
 
 1.00 
 
 
 Slope on short rate, Vasicek and OLS 
 
 
 maturity (years) 
 
 
 
 
 
 
 
 Vasicek 
 
 
 
 OLS